In [ ]:
import csv
import heapq
import itertools
import math
import textwrap
from difflib import SequenceMatcher

def load_movies_db(filename):
    movies_db = []
    with open(filename, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            movies_db.append(row)
    return movies_db

def get_similar_movies(movie_name, movies_db):
    # Search algorithm 1: Exact match
    exact_match = [movie for movie in movies_db if movie["title"].lower() == movie_name.lower()]
    
    # Search algorithm 2: Partial match
    state_space = [movie for movie in movies_db if movie_name.lower() in movie["title"].lower()]
    
    # Search algorithm 3: Levenshtein distance
    generate_and_test = heapq.nsmallest(6, movies_db, key=lambda movie: SequenceMatcher(None, movie["title"].lower(), movie_name.lower()).ratio())
    
    # Search algorithm 4: Jaccard similarity
    def jaccard_similarity(movie1, movie2):
        words1 = set(movie1["title"].lower().split())
        words2 = set(movie2["title"].lower().split())
        intersection = words1 & words2
        union = words1 | words2
        if len(union) == 0:
            return 0
        else:
            return len(intersection) / len(union)
    
    jaccard_similarities = [(movie, jaccard_similarity(movie, {"title": movie_name})) for movie in movies_db]
    jaccard_similarities.sort(key=lambda x: x[1], reverse=True)
    jaccard_similarity_results = [movie for movie, _ in jaccard_similarities[:6]]
    
    # Search algorithm 5: Cosine similarity
    def cosine_similarity(movie1, movie2):
        words1 = set(movie1["title"].lower().split())
        words2 = set(movie2["title"].lower().split())
        common_words = words1 & words2
        freq1 = [movie1["title"].lower().count(word) for word in common_words]
        freq2 = [movie2["title"].lower().count(word) for word in common_words]
        dot_product = sum(f1 * f2 for f1, f2 in zip(freq1, freq2))
        magnitude1 = math.sqrt(sum(f ** 2 for f in freq1))
        magnitude2 = math.sqrt(sum(f ** 2 for f in freq2))
        if magnitude1 == 0 or magnitude2 == 0:
            return 0
        else:
            return dot_product / (magnitude1 * magnitude2)
    
    cosine_similarities = [(movie, cosine_similarity(movie, {"title": movie_name})) for movie in movies_db]
    cosine_similarities.sort(key=lambda x: x[1], reverse=True)
    cosine_similarity_results = [movie for movie, _ in cosine_similarities[:6]]

    # Combine results from all search algorithms
    results = list(itertools.chain(exact_match, state_space, generate_and_test, jaccard_similarity_results, cosine_similarity_results))
    # Remove duplicates
    results = [dict(t) for t in {tuple(d.items()) for d in results}]
    
    return results

# Load movie database from CSV file
movies_db = load_movies_db('tmdb_5000_movies.csv')

movie = input('Enter Movie Name: ')

# Find similar movies
similar_movies = get_similar_movies(movie, movies_db)

wrapper = textwrap.TextWrapper(width=140)
# Print results
for movie in similar_movies:
    print("\n")
    
    print("Recommended Movie:", movie['title'], "\nRating:", movie['vote_average'])
    
    word_list = wrapper.wrap(text=movie['overview'])
    for element in word_list:
        print(element)
